<a href="https://colab.research.google.com/github/Mahonry/Fraud-Detection/blob/master/Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import numpy as np

In [0]:
df = pd.read_excel('Dataset.xlsx')
df.head()
#https://www.kaggle.com/incarnyx/car-insurance-fraud

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,FraudFound_P,PolicyNumber,RepNumber,Deductible,DriverRating,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy,ClaimSize
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,21.0,Policy Holder,Sport - Liability,Sport,more than 69000,0,1,12,300,1.0,more than 30,more than 30,none,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability,55526.069421
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,34.0,Policy Holder,Sport - Collision,Sport,more than 69000,0,2,15,400,4.0,more than 30,more than 30,none,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision,59294.455571
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,47.0,Policy Holder,Sport - Collision,Sport,more than 69000,0,3,7,400,3.0,more than 30,more than 30,1,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision,71756.941320
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,65.0,Third Party,Sedan - Liability,Sport,20000 to 29000,0,4,4,400,2.0,more than 30,more than 30,1,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability,7584.153493
4,Feb,1,Saturday,Honda,Urban,Monday,Feb,3,Male,Married,36.0,Third Party,Sport - Collision,Sport,more than 69000,0,7,14,400,1.0,more than 30,more than 30,1,7 years,36 to 40,No,No,External,1 to 2,no change,1 vehicle,1994,Collision,98545.374339


In [0]:
print(len(df))
print(len(df['FraudFound_P']))

11565
11565


# Inspección de Variables y preprocesado de datos


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11565 entries, 0 to 11564
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Month                 11565 non-null  object 
 1   WeekOfMonth           11565 non-null  int64  
 2   DayOfWeek             11565 non-null  object 
 3   Make                  11565 non-null  object 
 4   AccidentArea          11565 non-null  object 
 5   DayOfWeekClaimed      11565 non-null  object 
 6   MonthClaimed          11565 non-null  object 
 7   WeekOfMonthClaimed    11565 non-null  int64  
 8   Sex                   11565 non-null  object 
 9   MaritalStatus         11565 non-null  object 
 10  Age                   11560 non-null  float64
 11  Fault                 11565 non-null  object 
 12  PolicyType            11565 non-null  object 
 13  VehicleCategory       11565 non-null  object 
 14  VehiclePrice          11565 non-null  object 
 15  FraudFound_P       

In [0]:
#Observando en donde faltan valores
df.isna().sum()  != 0

Month                   False
WeekOfMonth             False
DayOfWeek               False
Make                    False
AccidentArea            False
DayOfWeekClaimed        False
MonthClaimed            False
WeekOfMonthClaimed      False
Sex                     False
MaritalStatus           False
Age                      True
Fault                   False
PolicyType              False
VehicleCategory         False
VehiclePrice            False
FraudFound_P            False
PolicyNumber            False
RepNumber               False
Deductible              False
DriverRating             True
Days_Policy_Accident    False
Days_Policy_Claim       False
PastNumberOfClaims      False
AgeOfVehicle            False
AgeOfPolicyHolder       False
PoliceReportFiled       False
WitnessPresent          False
AgentType               False
NumberOfSuppliments     False
AddressChange_Claim     False
NumberOfCars            False
Year                    False
BasePolicy              False
ClaimSize 

In [0]:
# Rellenando los valores faltantes
df['Age'] = df['Age'].fillna(df['Age'].mean()) 
df['DriverRating'] = df['DriverRating'].fillna(df['DriverRating'].mean())

La variable objetivo que clasifica la actividad como fraudulenta  es denominada "FraudFound_P", evaluaremos la distribución de esta variable:

In [0]:
print("Distribución de FraudFound: \n", 100*df["FraudFound_P"].value_counts()/df.shape[0])
print(len(df["FraudFound_P"]))

Distribución de FraudFound: 
 0    94.076956
1     5.923044
Name: FraudFound_P, dtype: float64
11565


Evaluemos las variables que tienen tipo object y la cantidad de valores distintos que poseen
---



In [0]:
df.dtypes.value_counts()

object     24
int64       7
float64     3
dtype: int64

In [0]:
# En la variable 'PastNumberOfClaims' tenemos mezclados tipos de datos por lo que convertire todos a texto para poder realizar el OneHotEncoding
df['PastNumberOfClaims'] = df['PastNumberOfClaims'].astype('category')
df.drop(1146, inplace=True)

In [0]:
types = pd.DataFrame(df.dtypes)
categoricas = types.index[types[0] == 'O'].values
for line in categoricas:
 print("La variable "+ line +" contiene:",str(len(df[line].unique()))+" distinct values")

La variable Month contiene: 12 distinct values
La variable DayOfWeek contiene: 7 distinct values
La variable Make contiene: 19 distinct values
La variable AccidentArea contiene: 2 distinct values
La variable DayOfWeekClaimed contiene: 7 distinct values
La variable MonthClaimed contiene: 12 distinct values
La variable Sex contiene: 2 distinct values
La variable MaritalStatus contiene: 4 distinct values
La variable Fault contiene: 2 distinct values
La variable PolicyType contiene: 9 distinct values
La variable VehicleCategory contiene: 3 distinct values
La variable VehiclePrice contiene: 6 distinct values
La variable Days_Policy_Accident contiene: 5 distinct values
La variable Days_Policy_Claim contiene: 3 distinct values
La variable AgeOfVehicle contiene: 8 distinct values
La variable AgeOfPolicyHolder contiene: 9 distinct values
La variable PoliceReportFiled contiene: 2 distinct values
La variable WitnessPresent contiene: 2 distinct values
La variable AgentType contiene: 2 distinct val

In [0]:
#Para la variables pueden tomar valores binarios, hacemos una sustitucion simple
df['AccidentArea'] = df['AccidentArea'].replace({'Urban': 0, 'Rural': 1})
df['Sex'] = df['Sex'].replace({'Female': 0, 'Male': 1})
df['Fault'] = df['Fault'].replace({'Policy Holder': 0, 'Third Party': 1})
df['PoliceReportFiled'] = df['PoliceReportFiled'].replace({'No': 0, 'Yes': 1})
df['WitnessPresent'] = df['WitnessPresent'].replace({'No': 0, 'Yes': 1})
df['AgentType'] = df['AgentType'].replace({'External': 0, 'Internal': 1})

In [0]:
print(len(df))
print(len(df['FraudFound_P']))

11564
11564


In [0]:
#Se define función que captura el dataframe y la columna para retornar un dataframe luego de One Hot Encoding
def crea_OneHotEncoding(df, columna):
  le = LabelEncoder()
  le_ajustado=le.fit_transform(df[columna]).reshape(-1,1)
  encoder = OneHotEncoder(sparse=False)
  column = [columna+ "_"+ str(i) for i in le.classes_]
  data = encoder.fit_transform(le_ajustado)
  return(pd.DataFrame(data,columns =column))

In [0]:
numeric_columns_1 = list(set(types.index[types[0] =="int64"].values))
numeric_columns_2 = list(set(types.index[types[0] =="float64"].values))
numeric_columns = numeric_columns_1 + numeric_columns_2
df_consolidado = df[numeric_columns]

In [0]:
categoricas = types.index[types[0] == 'O'].values
for column in categoricas:
  df_t = crea_OneHotEncoding(df,column)
  df_consolidado = pd.concat([df_consolidado,df_t],axis=1)
  print("Columna ",column, " transformada!")

print("Tamaño final del dataset transformado:",df_consolidado.shape)

Columna  Month  transformada!
Columna  DayOfWeek  transformada!
Columna  Make  transformada!
Columna  AccidentArea  transformada!
Columna  DayOfWeekClaimed  transformada!
Columna  MonthClaimed  transformada!
Columna  Sex  transformada!
Columna  MaritalStatus  transformada!
Columna  Fault  transformada!
Columna  PolicyType  transformada!
Columna  VehicleCategory  transformada!
Columna  VehiclePrice  transformada!
Columna  Days_Policy_Accident  transformada!
Columna  Days_Policy_Claim  transformada!
Columna  AgeOfVehicle  transformada!
Columna  AgeOfPolicyHolder  transformada!
Columna  PoliceReportFiled  transformada!
Columna  WitnessPresent  transformada!
Columna  AgentType  transformada!
Columna  NumberOfSuppliments  transformada!
Columna  AddressChange_Claim  transformada!
Columna  NumberOfCars  transformada!
Columna  BasePolicy  transformada!
Tamaño final del dataset transformado: (11565, 143)


In [0]:
# Hay un problema con el registro 1146 en la variable DayOfWeekClaimed, como es el unico registro que tiene este problema he decidido eliminarlo, y hay algunos valores nulos
# quitar estos valores nulos no representa perdida de información ya que solo estan dentro de un registro
df_consolidado.drop(1146, inplace=True)
df_consolidado.dropna(inplace=True)

In [0]:
objetivo = df_consolidado["FraudFound_P"]
df_consolidado.drop("FraudFound_P",axis = 1, inplace = True)

In [0]:
print(len(objetivo))
print(len(df_consolidado))

11563
11563


# Vamos a proponer dos modelos, un BaggingClassifier que tenga de base un algoritmo de KNeighbors y  un Random Forest



In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

bag_class = BaggingClassifier(base_estimator = KNeighborsClassifier(),
                                  n_estimators = 50)
forest = RandomForestClassifier(50)

models = [bag_class,forest]

# Evaluamos nuestros modelos

In [0]:
for i in models:
  results = cross_validate(i,
                         df_consolidado,
                         objetivo,
                         cv = 10,
                         return_train_score=True,)
  print('Model: {}'.format(str(i).split('(')[0]))
  test_scores = results['test_score']
  train_scores = results['train_score']
  print('Test score: ',np.mean(test_scores))
  print('Train score: ',np.mean(train_scores)) 
  print('___'*64)

Model: BaggingClassifier
Test score:  0.8777896989290405
Train score:  0.9425946635771865
________________________________________________________________________________________________________________________________________________________________________________________________
Model: RandomForestClassifier
Test score:  0.8995321841177368
Train score:  0.998971808936906
________________________________________________________________________________________________________________________________________________________________________________________________


Como conclusión final creo que nuestro mejor estimador es el BaggingClassifier ya que tuve un buen desempeño tanto en el entrenamiento como en la validacion, considero que el Random Forest podría estar cayendo en Overfitting, quiero remarcar que estos datos fueron tomados de una fuente pública, no conozco mayor descripción, veracidad y calidad de ellos, tan solo me limite a entender como estaban compuestos y a prepararlos para implementarlos dentro de estos dos modelos, aún podriamos mejorar nuestras estimaciones, recalculando hiperparámetros o incluso ampliando nuestro set de datos.